# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../cities1.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,makurdi,7.7411,8.5121,22.92,91,32,0.48,NG,2024-06-11
1,1,port-aux-francais,-49.3500,70.2167,1.68,65,89,17.10,TF,2024-06-11
2,2,pangody,65.8500,74.4859,9.00,78,67,3.80,RU,2024-06-11
3,3,bokoro,12.3807,17.0568,30.33,51,94,3.33,TD,2024-06-11
4,4,yeniseysk,58.4497,92.1703,13.30,79,17,1.34,RU,2024-06-11


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
coordinates = {
    "Lat": (city_data_df["Lat"]),
    "Lng": (city_data_df["Lng"]),
    "City": (city_data_df["City"]),
    "Humidity": (city_data_df["Humidity"])}

coordinates_df = pd.DataFrame(coordinates)

# Display the map
humidity_map = coordinates_df.hvplot.points("Lng","Lat", geo = True, tiles = "OSM",
    frame_width = 700, frame_height = 500, size = "Humidity", color = "City")

humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities_df = city_data_df.loc[(city_data_df['Max Temp']<=25)\
                            &(city_data_df['Max Temp']>=10)\
                            &(city_data_df['Cloudiness']<=60)\
                            &(city_data_df['Humidity']<70),:]
print(len(filtered_cities_df))

# Drop any rows with null values
filtered_cities_df.dropna()

# Display sample data
filtered_cities_df.head()

49


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,recani,31.2846,-4.2688,24.88,39,0,8.73,MA,2024-06-11
38,38,llerena,38.2354,-6.0739,18.01,64,56,2.79,ES,2024-06-11
41,41,ust-nera,64.5667,143.2000,14.83,43,45,1.57,RU,2024-06-11
43,43,masterton,-40.9597,175.6575,11.60,68,52,0.86,NZ,2024-06-11
45,45,luau,-10.7073,22.2247,13.42,52,17,2.16,AO,2024-06-11


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""
# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
10,recani,MA,31.2846,-4.2688,39,
38,llerena,ES,38.2354,-6.0739,64,
41,ust-nera,RU,64.5667,143.2000,43,
43,masterton,NZ,-40.9597,175.6575,68,
45,luau,AO,-10.7073,22.2247,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
recani - nearest hotel: Hotel Merzouga
llerena - nearest hotel: Gallego
ust-nera - nearest hotel: гостиница "Солнечная"
masterton - nearest hotel: Copthorne Hotel Solway Park
luau - nearest hotel: No hotel found
anadyr - nearest hotel: Гостевой дом
port hedland - nearest hotel: The Esplanade Hotel
emerald - nearest hotel: Emerald Central Hotel
karratha - nearest hotel: Karratha International Hotel
lubango - nearest hotel: Novo Hotel
guerrero negro - nearest hotel: Plaza sal paraiso
tshabong - nearest hotel: No hotel found
zag - nearest hotel: No hotel found
kone - nearest hotel: Pacifik Appartels
pavlovsk - nearest hotel: Гостиница "Дон"
nouadhibou - nearest hotel: Hotel valencia
kruisfontein - nearest hotel: No hotel found
labrador city - nearest hotel: Carol Inn
opuwo - nearest hotel: Ohakane Lodge
necochea - nearest hotel: Cabañas Río Quequén
santa luzia - nearest hotel: Alcoboca
ain oussera - nearest hotel: El Mahdi
happy valley-goose bay - nearest hotel: Hote

,City,Country,Lat,Lng,Humidity,Hotel Name
10,recani,MA,31.2846,-4.2688,39,Hotel Merzouga
38,llerena,ES,38.2354,-6.0739,64,Gallego
41,ust-nera,RU,64.5667,143.2000,43,"гостиница ""Солнечная"""
43,masterton,NZ,-40.9597,175.6575,68,Copthorne Hotel Solway Park
45,luau,AO,-10.7073,22.2247,52,No hotel found
46,anadyr,RU,64.7500,177.4833,66,Гостевой дом
79,port hedland,AU,-20.3167,118.5667,58,The Esplanade Hotel
83,emerald,AU,-23.5333,148.1667,33,Emerald Central Hotel
101,karratha,AU,-20.7377,116.8463,54,Karratha International Hotel
107,lubango,AO,-14.9172,13.4925,47,Novo Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", "Lat", geo = True, color = "City", alpha = 0.75,
                                   size = "Humidity", tiles = "OSM", hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)